# Parte 1: obtenemos los datos

In [416]:
import numpy as np
import deep_inv_opt as io
import deep_inv_opt.plot as iop
import torch

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.max_open_warning'] = 0  # Let the plots flow!
%matplotlib inline

In [417]:
u_train = io.tensor(np.linspace(0.5, 50, 1024).reshape((-1, 1)))
u_train

tensor([[ 0.5000],
        [ 0.5484],
        [ 0.5968],
        ...,
        [49.9032],
        [49.9516],
        [50.0000]], dtype=torch.float64)

In [418]:
u_val = io.tensor(np.linspace(0.5, 50, 256).reshape((-1, 1)))
u_val

tensor([[ 0.5000],
        [ 0.6941],
        [ 0.8882],
        [ 1.0824],
        [ 1.2765],
        [ 1.4706],
        [ 1.6647],
        [ 1.8588],
        [ 2.0529],
        [ 2.2471],
        [ 2.4412],
        [ 2.6353],
        [ 2.8294],
        [ 3.0235],
        [ 3.2176],
        [ 3.4118],
        [ 3.6059],
        [ 3.8000],
        [ 3.9941],
        [ 4.1882],
        [ 4.3824],
        [ 4.5765],
        [ 4.7706],
        [ 4.9647],
        [ 5.1588],
        [ 5.3529],
        [ 5.5471],
        [ 5.7412],
        [ 5.9353],
        [ 6.1294],
        [ 6.3235],
        [ 6.5176],
        [ 6.7118],
        [ 6.9059],
        [ 7.1000],
        [ 7.2941],
        [ 7.4882],
        [ 7.6824],
        [ 7.8765],
        [ 8.0706],
        [ 8.2647],
        [ 8.4588],
        [ 8.6529],
        [ 8.8471],
        [ 9.0412],
        [ 9.2353],
        [ 9.4294],
        [ 9.6235],
        [ 9.8176],
        [10.0118],
        [10.2059],
        [10.4000],
        [10.

ahora generamos los x correspondientes del modelo real

In [419]:
class ExamplePLP(io.ParametricLP):
    # Generate an LP from a given feature vector u and weight vector w.
    def generate(self, u, w):
        c = [[torch.cos(w + u**2 / 2)],
             [torch.sin(w + u**2 / 2)]]

        A_ub = [[-1.0,  0.0],      # x1 >= 0
                [ 0.0, -1.0],      # x2 >= 0
                [ 1.0,  0.0],      # x1 <= 2*w
                [ .5*w, w]]  # (1+w)*x1 + 2*(1+w)*x2 <= u

        b_ub = [[ 0.0],
                [ 0.0],
                [ 4/u],
                [   u]]
        
        return c, A_ub, b_ub, None, None

In [420]:
plp_true = ExamplePLP(weights=[0.8])

# Generate training targets by solve the true PLP at each u value.
# x_train = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u_train])
# torch.save(x_train, "x_train.pt")

# x_val = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u_train])
# torch.save(x_val, "x_val.pt")

In [421]:
x_val = torch.load("x_val.pt")
x_val

tensor([[6.3378e-06, 4.7761e-06],
        [6.4988e-06, 4.6438e-06],
        [6.1619e-06, 4.1501e-06],
        ...,
        [8.0112e-02, 1.4527e-05],
        [1.9709e-04, 6.2439e+01],
        [2.5363e-05, 5.3045e-05]], dtype=torch.float64)

In [422]:
x_train = torch.load("x_train.pt")
x_train

tensor([[6.3378e-06, 4.7761e-06],
        [6.4988e-06, 4.6438e-06],
        [6.1619e-06, 4.1501e-06],
        ...,
        [8.0112e-02, 1.4527e-05],
        [1.9709e-04, 6.2439e+01],
        [2.5363e-05, 5.3045e-05]], dtype=torch.float64)

# Parte 2: definimos la red y la entrenamos

In [423]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [424]:
# ver si estoy usando GPU
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [425]:
learning_rate = 1e-3
batch_size = 8
epochs = 100
tol=1

In [426]:
# definimos el dataset
class UDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data.clone().to(dtype=torch.float32) # nota: esta dando el warning porque estoy convirtiendo un tensor a otro, en ese caso es mejor usar clone()
        # si los datos de entrada no los voy a dar como un tensor, entonces hay que poner lo que he puesto: self.data = torch.tensor(data, dtype=torch.float32), self.targets = torch.tensor(targets, dtype=torch.float32)
        self.targets = targets.clone().to(dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# Dataset con pares (u, x)
dataset = UDataset(u_train, x_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [427]:
val_dataset = UDataset(u_val, x_val)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [428]:
# funcion que resuelve el problema de programacion lineal (por ahora nos la creemos, pero hay que revisarla)
# hay que tener en cuenta que esta funcion debe realizarse con operaciones de pytorch para
# preservar el grafo de computo para poder hacer backpropagation
# def smooth_lp(c, A, b):
#     # Inicializar x con gradientes habilitados
#     x = torch.zeros(A.shape[1], requires_grad=True)
#     optimizer = torch.optim.SGD([x], lr=1e-3)

#     for _ in range(1000):
#         optimizer.zero_grad()
#         constraint_penalty = torch.sum(torch.relu(A @ x - b))
#         objective = torch.dot(c, x) + 100.0 * constraint_penalty
#         objective.backward(retain_graph=True)  # Mantén el grafo activo
#         optimizer.step()
#     return x  # Sin detach()


In [441]:
# voy a definir otra funcion para resolver el problema de programacion lineal, pero esta vez con el solver de pytorch
import scipy.optimize as opt

def smooth_lp(c, A, b):
    c = c.detach().cpu().numpy()
    A = A.detach().cpu().numpy()
    b = b.detach().cpu().numpy()

    res = opt.linprog(c, A_ub=A, b_ub=b, method='highs-ipm')
    print(res)
    return torch.tensor(res.x, dtype=torch.float32)

# def smooth_lp(c, A, b):
#     return torch.tensor(opt.linprog(c, A_ub=A, b_ub=b, method='highs-ipm').x, dtype=torch.float32)

In [442]:
c = torch.tensor([1.0, 1.0])
A = torch.tensor([[-1.0, 0.0],
                  [0.0, -1.0]])
b = torch.tensor([0.0, 0.0])

In [443]:
solve_lp(c, A, b)

tensor([0., 0.])

Esta version es la red neuronal en la que la resolucion del problema de optimizacion esta dentro de la red

In [444]:
# definimos la red (hay que revisar la forma de la red y el por qué)
class ParametricLPNet(nn.Module):
    def __init__(self):
        super(ParametricLPNet, self).__init__()
        # Entrada de dimensión 1, salida 8 (2 para c, 4 para A, 2 para b)
        self.fc = nn.Sequential(
            nn.Linear(1, 8),
            nn.ReLU(),
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 8)  # c (2), A (4), b (2)
        )

    def forward(self, u):
        output = self.fc(u)
        c = output[:, 0:2]      # Vector de costes
        A = output[:, 2:6].reshape(-1, 2, 2)  # Matriz A (2x2)
        b = output[:, 6:8]      # Vector de restricciones
        return torch.stack([smooth_lp(c[i], A[i], b[i]) for i in range(u.shape[0])]) # Resolver LPs ver si hay alguna manera mas eficiente de hacer esto


In [445]:
# funcion de perdida
# def loss_fn(rs, target):
#     '''
#     rs: lista de tensores que son las salidas de la red
#     target: lista de tensores que son las salidas deseadas

#     Esta función calcula la suma de las distancias al cuadrado entre las salidas de la red y las salidas deseadas.
#     '''
#     return torch.sum(torch.linalg.vector_norm(rs-target, ord=2, dim=1).pow(2))

loss_fn = nn.MSELoss()

In [446]:
# Crear la red neuronal
model = ParametricLPNet()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # elegir una de las dos

In [447]:
# para ver el modelo
model

ParametricLPNet(
  (fc): Sequential(
    (0): Linear(in_features=1, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=8, bias=True)
  )
)

In [448]:
# inicializamod los pesos de la red 
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='leaky_relu') # inicializamos los pesos de la red con Kaiming
        nn.init.zeros_(m.bias) # inicializamos los bias a 0

initialize_weights(model)

In [449]:
def near(rs, x_batch, tol=1e-3):
    '''
    rs: tensor formado por las soluciones de los LPs
    x_batch: tensor formado por las soluciones deseadas de los LPs
    tol: tolerancia para considerar que dos vectores son iguales

    Esta función calcula el número de soluciones de los LPs que están 
    a una distancia menor que tol de las soluciones deseadas.
    '''
    matriz_diferencias = rs - x_batch # matriz de diferencias entre las soluciones de los LPs y las soluciones deseadas
    matriz_al_cuadrado = matriz_diferencias ** 2 # elevo cada elemento al cuadrado
    distancias = torch.sum(matriz_al_cuadrado, dim=1) # sumo los elementos de cada fila
    return torch.sum(distancias < tol).item()

In [450]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    loss_media = 0.0
    accuracy = 0.0
    batch_size = dataloader.batch_size
    for batch, (u_batch, x_batch) in enumerate(dataloader):

        rs = model(u_batch)
        loss = loss_fn(rs, x_batch) # Calcular la pérdida

        # Backpropagation y optimización
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 8 == 0: # cambiar este numero para que salga cada cierto numero de iteraciones
            loss = loss.item()
            current = batch * batch_size + batch_size
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
        loss_media += loss
        accuracy += near(rs, x_batch, tol)

    return loss_media / num_batches, accuracy / size

In [451]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size=len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    #with torch.no_grad():
    for u_batch, x_batch in dataloader:
        rs = model(u_batch)
        test_loss += loss_fn(rs, x_batch).item()
        correct += near(rs, x_batch, tol) # consideramos correcto si se acerca a la solucion en la distancia euclidea 
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n\n")

In [452]:
# Entrenamiento
epochs = 4 # poner mas
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    loss, accuracy = train_loop(dataloader, model, loss_fn, optimizer)
    print(f"\nTraining Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {loss:>8f} \n")
    
    test_loop(val_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -0.02193313677876964
              x: [ 0.000e+00  1.830e-01]
            nit: 6
          lower:  residual: [ 0.000e+00  1.830e-01]
                 marginals: [ 1.345e-02  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  7.405e-02]
                 marginals: [-3.102e-01  0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0
       message: The problem is unbounded. (HiGHS Status 10: model_status is Unbounded; primal_status is At upper bound)
       success: False
        status: 3
           fun: None
             x: None
           nit: 1
         lower:  residual: None
                marginals: None
         upper:  resi

TypeError: must be real number, not NoneType

In [ ]:
torch.save(model.state_dict(), "model_weights.pth")

In [ ]:
model(torch.tensor([[1.0]]))

tensor([[-2.3920, -2.2971]], grad_fn=<StackBackward0>)

In [ ]:
u_test = torch.tensor(np.linspace(0.1, 4, 16).reshape((-1, 1)), dtype=torch.float64)
u_test

tensor([[0.1000],
        [0.3600],
        [0.6200],
        [0.8800],
        [1.1400],
        [1.4000],
        [1.6600],
        [1.9200],
        [2.1800],
        [2.4400],
        [2.7000],
        [2.9600],
        [3.2200],
        [3.4800],
        [3.7400],
        [4.0000]], dtype=torch.float64)

ahora probamos como funciona para un x nuevo

In [ ]:
x_test = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u_test])
x_test

tensor([[5.5031e-06, 5.2914e-06],
        [5.8791e-06, 5.0125e-06],
        [6.2946e-06, 4.1114e-06],
        [1.5288e-05, 6.1703e-06],
        [6.3201e-05, 7.6848e-06],
        [2.8571e+00, 7.7249e-06],
        [2.4096e+00, 8.3817e-06],
        [2.0833e+00, 1.4402e-05],
        [1.8349e+00, 1.8074e+00],
        [1.6393e+00, 2.2303e+00],
        [6.9978e-05, 3.3749e+00],
        [1.2749e-05, 3.7000e+00],
        [6.8515e-06, 4.0250e+00],
        [1.1909e-05, 1.8498e-05],
        [2.2888e-04, 1.3794e-05],
        [9.9999e-01, 1.9565e-05]], dtype=torch.float64)

In [ ]:
model.eval()

ParametricLPNet(
  (fc): Sequential(
    (0): Linear(in_features=1, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=8, bias=True)
  )
)

In [ ]:
u = torch.tensor([[0.25]], dtype=torch.float64)
x = torch.cat([io.linprog(*plp_true(ui)).detach().t() for ui in u])
u,x

(tensor([[0.2500]], dtype=torch.float64),
 tensor([[5.6596e-06, 5.1630e-06]], dtype=torch.float64))

In [ ]:
model(torch.tensor([[0.25]]))

tensor([[-0.9870, -0.9380]], grad_fn=<StackBackward0>)

In [ ]:
x_pred = model(torch.tensor([[0.25]]))
x_pred

tensor([[-0.9870, -0.9380]], grad_fn=<StackBackward0>)

In [ ]:
x, x_pred, torch.norm(x - x_pred)

(tensor([[5.6596e-06, 5.1630e-06]], dtype=torch.float64),
 tensor([[-0.9870, -0.9380]], grad_fn=<StackBackward0>),
 tensor(1.3617, dtype=torch.float64, grad_fn=<LinalgVectorNormBackward0>))

Revisar:
- Función de pérdida ✔️
- Función smooth_lp (a ver si puedo sustituir la funcion smooth_lp por la funcion de linprog de deep_inv_opt)
- Organizar y explicar el código
- Estudiar por qué no disminuye el loss cuando entreno el modelo: es porque está mostrando el loss en el conjunto de test y no en el de entrenamiento ✔️
- Entrenar el modelo con más épocas y datos
- Pensar qué optimizador es mejor ✔️
- Pensar qué estructura de la red es mejor ✔️
- Inicialización de los pesos de la red ✔️
- Arreglar el porcentaje de aciertos ✔️



Bitácora: ahora lo que pasa es que da error porque el problema de optimización no está acotado, lo que tengo que hacer es revisar en el paper el intervalo donde se mueve la u porque creo que lo he puesto mal. Cuando arregle esto, tengo que volver a entrenar el modelo y ver si se está entrenando bien. También tengo que probar resolviendo el problema de optimización con la función de linprog de deep_inv_opt y con la de cvxpy a ver si alguna de las dos funciona mejor.